<a href="https://colab.research.google.com/github/awaiskhan005/DATA-SCIENCE-AND-AI-/blob/main/evaluate(3)(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install llama_index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 79.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.7 MB/s eta 0:00:00


In [3]:
import nest_asyncio

nest_asyncio.apply()

from llama_index.core.evaluation import generate_question_context_pairs
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core.evaluation import generate_question_context_pairs
from llama_index.core.evaluation import RetrieverEvaluator
from  llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI

import os
import pandas as pd

os.environ['OPENAI_API_KEY'] =''
os.environ['OPENAI_API_BASE']="https://api.xty.app/v1"

In [ ]:
documents = SimpleDirectoryReader("/home/user3/Base_model/Qwen/NSFc").load_data()

In [ ]:
llm = OpenAI(model="gpt-4",temperature=0.7)

In [ ]:
node_parser = SimpleNodeParser.from_defaults(chunk_size=512)
nodes = node_parser.get_nodes_from_documents(documents)
vector_index = VectorStoreIndex(nodes)

## 重新生成问题

In [ ]:
qa_generate_prompt_tmpl = """\
上下文信息如下所示：

---------------------
{context_str}
---------------------

给定上下文信息而非先验知识。
仅根据以下查询生成问题。

你是申请国家自然科学基金的研究人员。你的任务是为即将到来的中文测验设置{num_questions_per_chunk}个问题。
整个测试中的问题应该多种多样,问题不应包含选项,必须用中文书写。表达必须简明扼要。也就是说当你作为申请人的时候你最想根据你自己的项目问这上面的内容什么问题。
不应超过25个汉字。诸如"根据", "依据","你"和其他标点符号不应使用。缩写可用于标题和专业术语。
"""
test_dataset = generate_question_context_pairs(
    nodes[:],
    llm=llm,
    num_questions_per_chunk=1,
    qa_generate_prompt_tmpl=qa_generate_prompt_tmpl
)

100%|██████████| 128/128 [06:12<00:00,  2.91s/it]


In [ ]:
from llama_index.core.embeddings import resolve_embed_model
from llama_index.embeddings.huggingface import HuggingFaceEmbedding


EMBEDDINGS = {
    "OpenAI": OpenAIEmbedding(),
    "Bert": HuggingFaceEmbedding(model_name="google-bert/bert-base-chinese"),
    "Bge-zh": HuggingFaceEmbedding(model_name='BAAI/bge-large-zh-v1.5', trust_remote_code=True),
    "finetuned-Bge" : resolve_embed_model("local:/home/user3/Base_model/Qwen/bge-retrieve-Finetuning_Model"),
}

No sentence-transformers model found with name google-bert/bert-base-chinese. Creating a new one with MEAN pooling.
/home/user3/.conda/envs/RAG/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/user3/.conda/envs/RAG/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [ ]:

from llama_index.core.postprocessor import SentenceTransformerRerank
RERANKERS = {
    "WithoutReranker": "None",
    "bge-reranker-base": SentenceTransformerRerank(model="BAAI/bge-reranker-base", top_n=5),
    "bge-reranker-large": SentenceTransformerRerank(model="BAAI/bge-reranker-large", top_n=5),
    "finetuned-Bge-reranker" : SentenceTransformerRerank(model="/home/user3/Base_model/Qwen/bge-rerank-Finetuning_Model1",top_n=5),
}

In [ ]:
def display_results(embedding_name, reranker_name, eval_results):
    """Display results from evaluate."""

    metric_dicts = []
    for eval_result in eval_results:
        metric_dict = eval_result.metric_vals_dict
        metric_dicts.append(metric_dict)

    full_df = pd.DataFrame(metric_dicts)

    hit_rate = full_df["hit_rate"].mean()
    mrr = full_df["mrr"].mean()

    metric_df = pd.DataFrame(
        {"Embedding": [embedding_name], "Reranker": [reranker_name], "hit_rate": [hit_rate], "mrr": [mrr]}
    )

    return metric_df

In [ ]:
# 评估mrr和hit
from llama_index.core.indices.query.schema import QueryBundle, QueryType
from typing import List
from llama_index.core.schema import NodeWithScore
results_df = pd.DataFrame()
from llama_index.core.retrievers import (
    BaseRetriever,
    VectorIndexRetriever,
)

# Loop over embeddings
for embed_name, embed_model in EMBEDDINGS.items():

    service_context = ServiceContext.from_defaults(llm=None, embed_model=embed_model)
    vector_index = VectorStoreIndex(nodes, service_context=service_context)
    vector_retriever = VectorIndexRetriever(index=vector_index, similarity_top_k = 10)


    # Loop over rerankers
    for rerank_name, reranker in RERANKERS.items():

        print(f"Running Evaluation for Embedding Model: {embed_name} and Reranker: {rerank_name}")

        # Define Retriever
        class CustomRetriever(BaseRetriever):
            """Custom retriever that performs both Vector search and Knowledge Graph search"""

            def __init__(
                self,
                vector_retriever: VectorIndexRetriever,
            ) -> None:
                """Init params."""

                self._vector_retriever = vector_retriever

            def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
                """Retrieve nodes given query."""

                retrieved_nodes = self._vector_retriever.retrieve(query_bundle)

                if reranker != 'None':
                    retrieved_nodes = reranker.postprocess_nodes(retrieved_nodes, query_bundle)
                else:
                    retrieved_nodes = retrieved_nodes[:5]

                return retrieved_nodes

            async def _aretrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
                """Asynchronously retrieve nodes given query.

                Implemented by the user.

                """
                return self._retrieve(query_bundle)

            async def aretrieve(self, str_or_query_bundle: QueryType) -> List[NodeWithScore]:
                if isinstance(str_or_query_bundle, str):
                    str_or_query_bundle = QueryBundle(str_or_query_bundle)
                return await self._aretrieve(str_or_query_bundle)

        custom_retriever = CustomRetriever(vector_retriever)

        retriever_evaluator = RetrieverEvaluator.from_metric_names(
            ["mrr", "hit_rate"], retriever=custom_retriever
        )

        eval_results = await retriever_evaluator.aevaluate_dataset(test_dataset)

        current_df = display_results(embed_name, rerank_name, eval_results)
        results_df = pd.concat([results_df, current_df], ignore_index=True)

/tmp/ipykernel_1708168/2359604037.py:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(llm=None, embed_model=embed_model)


LLM is explicitly disabled. Using MockLLM.
Running Evaluation for Embedding Model: OpenAI and Reranker: WithoutReranker
Running Evaluation for Embedding Model: OpenAI and Reranker: bge-reranker-base
Running Evaluation for Embedding Model: OpenAI and Reranker: bge-reranker-large
Running Evaluation for Embedding Model: OpenAI and Reranker: finetuned-Bge-reranker
LLM is explicitly disabled. Using MockLLM.
Running Evaluation for Embedding Model: Bert and Reranker: WithoutReranker
Running Evaluation for Embedding Model: Bert and Reranker: bge-reranker-base
Running Evaluation for Embedding Model: Bert and Reranker: bge-reranker-large
Running Evaluation for Embedding Model: Bert and Reranker: finetuned-Bge-reranker
LLM is explicitly disabled. Using MockLLM.
Running Evaluation for Embedding Model: Bge-zh and Reranker: WithoutReranker
Running Evaluation for Embedding Model: Bge-zh and Reranker: bge-reranker-base
Running Evaluation for Embedding Model: Bge-zh and Reranker: bge-reranker-large
Run

In [ ]:
print(results_df)

        Embedding                Reranker  hit_rate       mrr
0          OpenAI         WithoutReranker  0.757812  0.584896
1          OpenAI       bge-reranker-base  0.859375  0.682161
2          OpenAI      bge-reranker-large  0.875000  0.735156
3          OpenAI  finetuned-Bge-reranker  0.875000  0.717578
4            Bert         WithoutReranker  0.578125  0.394401
5            Bert       bge-reranker-base  0.703125  0.601172
6            Bert      bge-reranker-large  0.710938  0.633073
7            Bert  finetuned-Bge-reranker  0.703125  0.604036
8          Bge-zh         WithoutReranker  0.734375  0.530208
9          Bge-zh       bge-reranker-base  0.812500  0.667188
10         Bge-zh      bge-reranker-large  0.828125  0.717969
11         Bge-zh  finetuned-Bge-reranker  0.843750  0.680859
12  finetuned-Bge         WithoutReranker  0.851562  0.655599
13  finetuned-Bge       bge-reranker-base  0.882812  0.717578
14  finetuned-Bge      bge-reranker-large  0.914062  0.760547
15  fine

In [ ]:
results_df.to_csv('benchmark-final.csv', index=False)

In [ ]:
%%capture captured_output_ContextualRelevancy
## 评估AnswerRelevancy Faithfulness ContextualRelevancy
import pandas as pd

# 创建包含结果的DataFrame
results_df_ContextualRelevancy = pd.DataFrame(columns=["Embedding Model", "Rerank Model", "Average Contextual Relevancy Score"])

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from llama_index.core.indices.query.schema import QueryBundle, QueryType
from typing import List
from llama_index.core.schema import NodeWithScore
from llama_index.core.retrievers import (
    BaseRetriever,
    VectorIndexRetriever,
)
from llama_index.core.query_engine import RetrieverQueryEngine

from deepeval.integrations.llama_index import DeepEvalAnswerRelevancyEvaluator
from deepeval.integrations.llama_index import DeepEvalFaithfulnessEvaluator
from deepeval.integrations.llama_index import DeepEvalContextualRelevancyEvaluator

# Loop over embeddings
for embed_name, embed_model in EMBEDDINGS.items():

    service_context = ServiceContext.from_defaults(llm=None, embed_model=embed_model)
    vector_index = VectorStoreIndex(nodes, service_context=service_context)
    vector_retriever = VectorIndexRetriever(index=vector_index, similarity_top_k = 10)


    # Loop over rerankers
    for rerank_name, reranker in RERANKERS.items():

        print(f"Running Evaluation for Embedding Model: {embed_name} and Reranker: {rerank_name}")

        # Define Retriever
        class CustomRetriever(BaseRetriever):
            """Custom retriever that performs both Vector search and Knowledge Graph search"""

            def __init__(
                self,
                vector_retriever: VectorIndexRetriever,
            ) -> None:
                """Init params."""
                self._vector_retriever = vector_retriever

            def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
                """Retrieve nodes given query."""
                retrieved_nodes = self._vector_retriever.retrieve(query_bundle)
                if reranker != 'None':
                    retrieved_nodes = reranker.postprocess_nodes(retrieved_nodes, query_bundle)
                else:
                    retrieved_nodes = retrieved_nodes[:5]
                return retrieved_nodes

            async def _aretrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
                return self._retrieve(query_bundle)
            async def aretrieve(self, str_or_query_bundle: QueryType) -> List[NodeWithScore]:
                if isinstance(str_or_query_bundle, str):
                    str_or_query_bundle = QueryBundle(str_or_query_bundle)
                return self._aretrieve(str_or_query_bundle)

        custom_retriever = CustomRetriever(vector_retriever)
        query_engine = RetrieverQueryEngine.from_args(custom_retriever)
        # query_engine = custom_retriever.as_query_engine()
        queries_list = list(test_dataset.queries.values())

        contextual_relevancy_scores = []

        for query in queries_list:
            response_object =query_engine.query(query)
            contextual_relevancy_evaluator = DeepEvalContextualRelevancyEvaluator(
            threshold=0.5,
            model="gpt-4",
            include_reason=False)


            evaluation_result_contextual_relevancy = contextual_relevancy_evaluator.evaluate_response(
                query=query,
                response=response_object
            )

            contextual_relevancy_scores.append(evaluation_result_contextual_relevancy.score)
        # Calculate averages

        average_contextual_relevancy_score = sum(contextual_relevancy_scores) / len(contextual_relevancy_scores)

        current_df = pd.DataFrame({"Embedding Model": embed_name, "Rerank Model": rerank_name, "AverageContextualRelevancy_Score": average_contextual_relevancy_score}, index=[0])
        results_df_ContextualRelevancy  = pd.concat([results_df_ContextualRelevancy , current_df], ignore_index=True)



In [ ]:
print(results_df_ContextualRelevancy)

  Embedding Model            Rerank Model Average Contextual Relevancy Score  \
0          Bge-zh         WithoutReranker                                NaN   
1          Bge-zh       bge-reranker-base                                NaN   
2          Bge-zh      bge-reranker-large                                NaN   
3          Bge-zh  finetuned-Bge-reranker                                NaN   

   AverageContextualRelevancy_Score  
0                          0.777778  
1                          0.666667  
2                          0.777778  
3                          0.777778  


In [ ]:
results_df_ContextualRelevancy.to_csv('ContextualRelevancy.csv', index=False)

In [ ]:
%%capture captured_output_Faithfulness
## 评估Faithfulness
import pandas as pd

# 创建包含结果的DataFrame
results_df_Faithfulness = pd.DataFrame(columns=["Embedding Model", "Rerank Model", "Average_Faithfulness_Score"])

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from llama_index.core.indices.query.schema import QueryBundle, QueryType
from typing import List
from llama_index.core.schema import NodeWithScore
results_df = pd.DataFrame()
from llama_index.core.retrievers import (
    BaseRetriever,
    VectorIndexRetriever,
)
from llama_index.core.query_engine import RetrieverQueryEngine

from deepeval.integrations.llama_index import DeepEvalAnswerRelevancyEvaluator
from deepeval.integrations.llama_index import DeepEvalFaithfulnessEvaluator
from deepeval.integrations.llama_index import DeepEvalContextualRelevancyEvaluator

# Loop over embeddings
for embed_name, embed_model in EMBEDDINGS.items():

    service_context = ServiceContext.from_defaults(llm=None, embed_model=embed_model)
    vector_index = VectorStoreIndex(nodes, service_context=service_context)
    vector_retriever = VectorIndexRetriever(index=vector_index, similarity_top_k = 10)


    # Loop over rerankers
    for rerank_name, reranker in RERANKERS.items():

        print(f"Running Evaluation for Embedding Model: {embed_name} and Reranker: {rerank_name}")

        # Define Retriever
        class CustomRetriever(BaseRetriever):
            """Custom retriever that performs both Vector search and Knowledge Graph search"""

            def __init__(
                self,
                vector_retriever: VectorIndexRetriever,
            ) -> None:
                """Init params."""
                self._vector_retriever = vector_retriever

            def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
                """Retrieve nodes given query."""
                retrieved_nodes = self._vector_retriever.retrieve(query_bundle)
                if reranker != 'None':
                    retrieved_nodes = reranker.postprocess_nodes(retrieved_nodes, query_bundle)
                else:
                    retrieved_nodes = retrieved_nodes[:5]
                return retrieved_nodes

            async def _aretrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
                return self._retrieve(query_bundle)
            async def aretrieve(self, str_or_query_bundle: QueryType) -> List[NodeWithScore]:
                if isinstance(str_or_query_bundle, str):
                    str_or_query_bundle = QueryBundle(str_or_query_bundle)
                return self._aretrieve(str_or_query_bundle)

        custom_retriever = CustomRetriever(vector_retriever)
        query_engine = RetrieverQueryEngine.from_args(custom_retriever)
        # query_engine = custom_retriever.as_query_engine()
        queries_list = list(test_dataset.queries.values())

        faithfulness_scores = []

        for query in queries_list:
            response_object =query_engine.query(query)
            faithfulness_evaluator = DeepEvalFaithfulnessEvaluator(
            threshold=0.5,
            model="gpt-4",
            include_reason=False)

            evaluation_result_faithfulness = faithfulness_evaluator.evaluate_response(
                query=query,
                response=response_object
            )

            faithfulness_scores.append(evaluation_result_faithfulness.score)
        # Calculate averages
        average_faithfulness_score = sum(faithfulness_scores) / len(faithfulness_scores)

        current_df = pd.DataFrame({"Embedding Model": embed_name, "Rerank Model": rerank_name, "Average_Faithfulness_Score": average_faithfulness_score}, index=[0])
        results_df_Faithfulness = pd.concat([results_df_Faithfulness, current_df], ignore_index=True)

In [ ]:
print(results_df_Faithfulness)

  Embedding Model     Rerank Model  Average_Faithfulness_Score
0          OpenAI  WithoutReranker                         1.0


In [ ]:
results_df_Faithfulness.to_csv('Faithfulness.csv', index=False)

In [ ]:
%%capture captured_output_AnswerRelevancy

import pandas as pd

# 创建包含结果的DataFrame
results_df_AnswerRelevancy = pd.DataFrame(columns=["Embedding Model", "Rerank Model", "Average AnswerRelevancy Score"])

## 评估AnswerRelevancy
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from llama_index.core.indices.query.schema import QueryBundle, QueryType
from typing import List
from llama_index.core.schema import NodeWithScore
results_df = pd.DataFrame()
from llama_index.core.retrievers import (
    BaseRetriever,
    VectorIndexRetriever,
)
from llama_index.core.query_engine import RetrieverQueryEngine

from deepeval.integrations.llama_index import DeepEvalAnswerRelevancyEvaluator
from deepeval.integrations.llama_index import DeepEvalFaithfulnessEvaluator
from deepeval.integrations.llama_index import DeepEvalContextualRelevancyEvaluator

# Loop over embeddings
for embed_name, embed_model in EMBEDDINGS.items():

    service_context = ServiceContext.from_defaults(llm=None, embed_model=embed_model)
    vector_index = VectorStoreIndex(nodes, service_context=service_context)
    vector_retriever = VectorIndexRetriever(index=vector_index, similarity_top_k = 10)


    # Loop over rerankers
    for rerank_name, reranker in RERANKERS.items():

        print(f"Running Evaluation for Embedding Model: {embed_name} and Reranker: {rerank_name}")

        # Define Retriever
        class CustomRetriever(BaseRetriever):
            """Custom retriever that performs both Vector search and Knowledge Graph search"""

            def __init__(
                self,
                vector_retriever: VectorIndexRetriever,
            ) -> None:
                """Init params."""
                self._vector_retriever = vector_retriever

            def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
                """Retrieve nodes given query."""
                retrieved_nodes = self._vector_retriever.retrieve(query_bundle)
                if reranker != 'None':
                    retrieved_nodes = reranker.postprocess_nodes(retrieved_nodes, query_bundle)
                else:
                    retrieved_nodes = retrieved_nodes[:5]
                return retrieved_nodes

            async def _aretrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
                return self._retrieve(query_bundle)
            async def aretrieve(self, str_or_query_bundle: QueryType) -> List[NodeWithScore]:
                if isinstance(str_or_query_bundle, str):
                    str_or_query_bundle = QueryBundle(str_or_query_bundle)
                return self._aretrieve(str_or_query_bundle)

        custom_retriever = CustomRetriever(vector_retriever)
        query_engine = RetrieverQueryEngine.from_args(custom_retriever)
        # query_engine = custom_retriever.as_query_engine()
        queries_list = list(test_dataset.queries.values())

        answer_relevancy_scores = []
        faithfulness_scores = []
        contextual_relevancy_scores = []

        for query in queries_list:
            response_object =query_engine.query(query)

            answer_relevancy_evaluator = DeepEvalAnswerRelevancyEvaluator(
            threshold=0.5,
            model="gpt-4",
            include_reason=False)


            evaluation_result_answer_relevancy = answer_relevancy_evaluator.evaluate_response(
                query=query,
                response=response_object
            )
            answer_relevancy_scores.append(evaluation_result_answer_relevancy.score)

        # Calculate averages
        average_answer_relevancy_score = sum(answer_relevancy_scores) / len(answer_relevancy_scores)

        current_df = pd.DataFrame({"Embedding Model": embed_name, "Rerank Model": rerank_name, "Average AnswerRelevancy Score": average_answer_relevancy_score}, index=[0])

        results_df_AnswerRelevancy = pd.concat([results_df_AnswerRelevancy, current_df], ignore_index=True)


In [ ]:
print(results_df_AnswerRelevancy)

  Embedding Model     Rerank Model  Average AnswerRelevancy Score
0          OpenAI  WithoutReranker                            1.0


In [ ]:
results_df_AnswerRelevancy.to_csv('AnswerRelevancy.csv', index=False)